# Yugioh TCG Data Sources Merge
In this final notebook, we take the data and merge it together to create a view of the products and cards.

## Quickstart

1. Change filenames for data sources in the first code cell
2. Run all cells

In [20]:
# load in relevant files
import pandas as pd
import utils
#import importlib
#importlib.reload(utils)

cards_df = pd.read_csv('yugioh-index-expand-2025SEP05-111811.csv') # from yugioh-index-expand.ipynb
market_df = pd.read_csv('yugioh-tcgplayer-2025SEP05-185755.db.csv') # from yugioh-tcgplayer.ipynb

In [21]:
market_df.sample(10)

,Unnamed: 0,name,number,rarity,price,quantity,quantity_sellers,price_change_3m,volatility,index
22669,22669,Guardragon Reincarnation - Dark Neostorm (DANE),DANE-EN060,Common,$0.13,0,0,NaN,Indeterminate,189415
15859,15859,Constellar Kaus - Star Pack ARC-V (SP15),SP15-EN008,Common,$1.59,0,0,NaN,Indeterminate,99869
31248,31248,Gate Guardian of Thunder and Wind - Maze of Me...,MAZE-EN004,Super Rare,$0.45,0,0,NaN,Low,485508
44460,44460,Kelbek - Dark Crisis (Worldwide English) (DCR-EN),DCR-EN078,Common,-,0,0,NaN,NaN,476887
30353,30353,"Zep, Ruby of the Ghoti - Darkwing Blast (DABL)",DABL-EN085,Ultra Rare,$1.27,0,0,NaN,Low,450198
34359,34359,Libromancer Fire - 25th Anniversary Tin: Dueli...,MP23-EN058,Ultra Rare,$0.13,0,0,NaN,Low,514746
19205,19205,Odd-Eyes Lancer Dragon - Battles of Legend: Li...,BLLR-EN001,Ultra Rare,$0.24,0,0,NaN,Low,136790
25082,25082,Mathmech Diameter - Rise of the Duelist (ROTD),ROTD-EN028,Common,$0.67,0,0,NaN,Low,218904
8240,8240,Marauding Captain - Structure Deck: Warrior's ...,SD5-EN009,Common,$0.18,0,0,NaN,Indeterminate,24528
40653,40653,Token: Leo & Life Stream Dragon - Duelist's Ad...,TKN5-EN028,Super Rare,$0.51,0,0,NaN,Med,641891


In [22]:
cards_df.sample(10)

,Unnamed: 0,index,name,description,type,sub_type,attribute,rank,attack,defense,set_name,set_id,set_release,rarity
20216,20216,19404,Legendary Fire King Ponix,If your monster(s) that was originally FIRE is...,MONSTER,[Winged Beast／Effect],FIRE,Level 1,500,200,LIMITED PACK WORLD CHAMPIONSHIP 2025,25LP-EN004,2025-09-04,UR Ultra Rare
18897,18897,11364,"Jiaotu, Darkness of the Yang Zing",When this card you control is destroyed by bat...,MONSTER,[Wyrm／Tuner／Effect],DARK,Level 2,0,2000,2015 MEGA-TINS MEGA PACK,MP15-EN151,2015-09-18,UR Ultra Rare
7847,7847,6582,Damage Condenser,When you take battle damage: Discard 1 card; S...,TRAP,NaN,NaN,NaN,NaN,NaN,SHADOW OF INFINITY,SOI-EN052,2006-02-18,UL Ultimate Rare
21254,21254,11057,Madolche Anjelly,"You can Tribute this card; Special Summon 1 ""M...",MONSTER,[Fairy／Effect],EARTH,Level 4,1000,1000,PRIMAL ORIGIN,PRIO-EN028,2014-05-16,UR Ultra Rare
706,706,7827,Ally of Justice Unknown Crusher,"When this card battles a LIGHT monster, remove...",MONSTER,[Machine／Effect],DARK,Level 3,1200,800,DUEL TERMINAL 1,DT01-EN079,2010-01-29,R Rare
29992,29992,5027,Robotic Knight,"The Commander of Machine-Types, he serves the ...",MONSTER,[Machine／Normal],FIRE,Level 4,1600,1800,DEMO DECK 2015,DEM2-EN007,2015-03-30,C Common
16042,16042,10654,Googly-Eyes Drum Dragon,2 Level 8 Machine-Type monstersOnce per turn: ...,MONSTER,[Machine／Xyz／Effect],EARTH,Rank 8,3000,2500,JUDGMENT OF THE LIGHT,JOTL-EN051,2013-08-09,C Common
28639,28639,13485,Rainbow Bridge,"Add 1 ""Crystal"" Spell/Trap from your Deck to y...",SPELL,NaN,NaN,NaN,NaN,NaN,LEGENDARY DUELISTS: ANCIENT MILLENNIUM,LED2-EN038,2018-02-23,UR Ultra Rare
20908,20908,11760,Lucius the Shadow Vassal,You can banish 1 Level 5 or higher monster fro...,MONSTER,[Fiend／Effect],DARK,Level 1,800,1000,EMPEROR OF DARKNESS STRUCTURE DECK,SR01-EN010,2016-01-29,C Common
4282,4282,6690,Bountiful Artemis,"Each time a Counter Trap Card is activated, im...",MONSTER,[Fairy／Effect],LIGHT,Level 4,1600,1700,QUARTER CENTURY STAMPEDE,RA04-EN130,2025-04-11,PS Platinum Secret Rare


## Datasource Merging
In order to combine or merge multiple databases, we need to create what's called a foreign key. This is a value that can be used to cross reference across multiple different databases. The following cells are meant to create a foreign key using the card's set number and rarity. 

In [23]:
rare_lookup = {'Rare': 'R',
 'Gold Rare': 'GR',
 'Starfoil Rare': 'ST',
 '10000 Secret Rare': '10000 SE',
 'Platinum Secret Rare': 'PS',
 'Super Rare': 'SR',
 'Common': 'C',
 'Quarter Century Secret Rare': 'QCSE',
 'Shatterfoil Rare': 'SH',
 'Secret Rare': 'SE',
 'Prismatic Ultimate Rare': 'PSE',
 'Secret Pharaoh’s Rare': 'SE (PR)',
 'Ultra Rare': 'UR',
 'Premium Gold Rare': 'PGR',
 'Promo': None,
 'Ultra Pharaoh’s Rare': 'UR (PR)',
 "Prismatic Collector's Rare": 'PSE',
 'Platinum Rare': 'PL',
 'Ghost Rare': 'GR',
 'Parallel Rare': None,
 'Starlight Rare': 'STAR',
 'Ultimate Rare': 'UL',
 'Ghost/Gold Rare': 'GH',
 "Collector's Rare": 'CR',
 'Prismatic Secret Rare': 'PSE',
 'Mosaic Rare': 'MR',
 'Gold Secret Rare': 'GSE'
}

In [24]:
rarity_konami = {'10000 SE 10000 SECRET RARE': '10000 SE',
 'C Common': 'C',
 "CR COLLECTOR'S RARE": 'CR',
 'GH Ghost Rare': 'GH',
 'GR Gold Rare': 'GR',
 'GSE Gold Secret': 'GSE',
 'H Hobby': 'H',
 'MR Mosaic Rare': 'MR',
 'PGR Premium Gold Rare': 'PGR',
 'PL Platinum Rare': 'PL',
 'PS Platinum Secret Rare': 'PS',
 'PSE Prismatic Secret Rare': 'PSE',
 'QCSE Quarter Century Secret Rare': 'QCSE',
 'R Rare': 'R',
 'SE Secret Rare': 'SE',
 'SH Shatterfoil': 'SH',
 'SR Super Rare': 'SR',
 'ST Starfoil': 'ST',
 'STAR Starlight Rare': 'STAR',
 'UL Ultimate Rare': 'UL',
 'UR (Hobby) Ultra Rare（Hobby League Version）': 'UR (Hobby)',
 "UR (PR) Ultra Rare (Pharaoh's Rare)": 'UR (PR)',
 'UR Ultra Rare': 'UR',
 'UR Ultra Rare (Duelist Saga Version)': 'UR (Duelist Saga Version)'
}

In [25]:
[rarity for rarity in set(market_df['rarity']) if not pd.isnull(rarity) and 'Duel Terminal' not in rarity]

['Prismatic Secret Rare',
 'Common',
 'Platinum Secret Rare',
 '10000 Secret Rare',
 'Promo',
 'Emblazoned Ultra Rare',
 'Shatterfoil Rare',
 'Ultimate Rare',
 'Secret Rare',
 'Starfoil Rare',
 'Mosaic Rare',
 'Ultra Pharaoh’s Rare',
 'Platinum Rare',
 'Starlight Rare',
 'Parallel Rare',
 'Super Rare',
 'Premium Gold Rare',
 'Rare',
 'Secret Pharaoh’s Rare',
 "Prismatic Collector's Rare",
 'Emblazoned Secret Rare',
 'Prismatic Ultimate Rare',
 'Gold Rare',
 'Gold Secret Rare',
 'Ghost/Gold Rare',
 "Collector's Rare",
 'Ghost Rare',
 'Ultra Rare',
 'Quarter Century Secret Rare']

In [26]:
set(cards_df['rarity'])

{'10000 SE 10000 SECRET RARE',
 'C Common',
 "CR COLLECTOR'S RARE",
 'GH Ghost Rare',
 'GR Gold Rare',
 'GSE Gold Secret',
 'H Hobby',
 'MR Mosaic Rare',
 'PGR Premium Gold Rare',
 'PL Platinum Rare',
 'PS Platinum Secret Rare',
 'PSE Prismatic Secret Rare',
 'QCSE Quarter Century Secret Rare',
 'R Rare',
 'SE Secret Rare',
 'SH Shatterfoil',
 'SR Super Rare',
 'ST Starfoil',
 'STAR Starlight Rare',
 'UL Ultimate Rare',
 'UR (Hobby) Ultra Rare（Hobby League Version）',
 "UR (PR) Ultra Rare (Pharaoh's Rare)",
 'UR Ultra Rare',
 'UR Ultra Rare (Duelist Saga Version)'}

In [27]:
cards_df['join_id'] = cards_df.apply(lambda card: f"{card['set_id']}{rarity_konami.get(card['rarity'])}", axis=1)

In [28]:
market_df['join_id'] = market_df.apply(lambda card: f"{card['number']}{rare_lookup.get(card['rarity'])}", axis=1)

In [29]:
merged_df = cards_df.merge(market_df[['name', 'join_id', 'price', 'volatility', 'index']],
                           on='join_id',
                           suffixes=[None, '_market']
                          )

In [30]:
len(merged_df)

40792

In [31]:
# example spot check, usually for recently released
merged_df[merged_df['name'] == 'Purrely']

,Unnamed: 0,index,name,description,type,sub_type,attribute,rank,attack,defense,set_name,set_id,set_release,rarity,join_id,name_market,price,volatility,index_market
28405,28167,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy／Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,SR Super Rare,RA02-EN018SR,Purrely - 25th Anniversary Rarity Collection I...,$0.23,Low,550968
28406,28168,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy／Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,UR Ultra Rare,RA02-EN018UR,Purrely (UR) - 25th Anniversary Rarity Collect...,$0.15,Low,550969
28407,28169,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy／Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,SE Secret Rare,RA02-EN018SE,Purrely (Secret Rare) - 25th Anniversary Rarit...,$0.29,Low,550970
28408,28171,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy／Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,PS Platinum Secret Rare,RA02-EN018PS,Purrely (Platinum Secret Rare) - 25th Anniver...,$0.99,Med,550971
28409,28173,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy／Effect],LIGHT,Level 1,100,100,25TH ANNIVERSARY RARITY COLLECTION 2,RA02-EN018,2024-05-24,QCSE Quarter Century Secret Rare,RA02-EN018QCSE,Purrely (Quarter Century Secret Rare) - 25th A...,$7.93,Med,550972
28410,28174,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy／Effect],LIGHT,Level 1,100,100,AMAZING DEFENDERS,AMDE-EN013,2023-01-20,UR Ultra Rare,AMDE-EN013UR,Purrely - Amazing Defenders (AMDE),$1.76,Med,477702
28411,28175,18003,Purrely,If this card is Normal or Special Summoned: Yo...,MONSTER,[Fairy／Effect],LIGHT,Level 1,100,100,AMAZING DEFENDERS,AMDE-EN013,2023-01-20,CR COLLECTOR'S RARE,AMDE-EN013CR,Purrely (CR) - Amazing Defenders (AMDE),$18.47,Low,477703


In [32]:
merged_df['name_official'] = merged_df['name']
merged_df['name'] = merged_df.apply(lambda row: utils.card_name_compare(row), axis=1)
#merged_df[merged_df['name_compare'] != merged_df['name']][['name', 'name_market', 'name_compare']].to_csv('name_changes.csv')

In [33]:
output_df = merged_df
output_df.sample(10)

,Unnamed: 0,index,name,description,type,sub_type,attribute,rank,attack,defense,set_name,set_id,set_release,rarity,join_id,name_market,price,volatility,index_market,name_official
20130,19907,10732,Labradorite Dragon,The ancients compared this prized creature to ...,MONSTER,[Dragon／Tuner／Normal],DARK,Level 6,0,2400,SHADOW SPECTERS,SHSP-EN001,2013-11-08,SR Super Rare,SHSP-EN001SR,Labradorite Dragon - Shadow Specters (SHSP),$3.83,Med,72356,Labradorite Dragon
19494,19305,19023,Kaitoptera,If your opponent controls 2 or more face-up mo...,MONSTER,[Dinosaur／Effect],WIND,Level 4,1400,1000,MAZE OF MILLENNIA,MZMI-EN012,2024-01-19,SR Super Rare,MZMI-EN012SR,Kaitoptera - Maze of Millennia (MZMI),$0.63,Low,532217,Kaitoptera
1464,1507,9610,Aquamirror Illusion,"Special Summon 1 ""Gishki"" Ritual Monster from ...",TRAP,NaN,NaN,NaN,NaN,NaN,HIDDEN ARSENAL 6 OMEGA XYZ,HA06-EN059,2012-07-24,SR Super Rare,HA06-EN059SR,Aquamirror Illusion - Hidden Arsenal 6: Omega ...,-,NaN,60102,Aquamirror Illusion
13714,13601,11348,Fluffal Bear,You can send this card from your hand to the G...,MONSTER,[Fairy／Effect],EARTH,Level 3,1200,800,STAR PACK ARC-V,SP15-EN023,2015-06-12,C Common,SP15-EN023C,Fluffal Bear - Star Pack ARC-V (SP15),$0.60,Indeterminate,99873,Fluffal Bear
761,805,18856,Altergeist Revitalization,"Target 1 ""Altergeist"" Link Monster in your GY;...",TRAP,NaN,NaN,NaN,NaN,NaN,DUELIST NEXUS,DUNE-EN069,2023-07-28,C Common,DUNE-EN069C,Altergeist Revitalization - Duelist Nexus (DUNE),$0.12,Low,504919,Altergeist Revitalization
11714,11566,6653,Elemental HERO Neos,A new Elemental HERO has arrived from Neo-Spac...,MONSTER,[Warrior／Normal],LIGHT,Level 7,2500,2000,MAGNIFICENT MAVENS,MAMA-EN106,2022-11-04,UR (PR) Ultra Rare (Pharaoh's Rare),MAMA-EN106UR (PR),Elemental HERO Neos (Ultra Pharaoh's Rare) - M...,$146.00,Indeterminate,452549,Elemental HERO Neos
4045,3988,4007,Blue-Eyes White Dragon,This legendary dragon is a powerful engine of ...,MONSTER,[Dragon／Normal],LIGHT,Level 8,3000,2500,Speed Duel: Battle City Box,SBCB-EN087,2020-11-26,C Common,SBCB-EN087C,Blue-Eyes White Dragon - Speed Duel: Battle Ci...,$0.64,Low,227984,Blue-Eyes White Dragon
14053,13939,6198,Fox Fire,"During the End Phase, if this card was destroy...",MONSTER,[Pyro／Effect],FIRE,Level 2,300,200,STRUCTURE DECK BLAZE OF DESTRUCTION,SD3-EN013,2005-05-09,C Common,SD3-EN013C,Fox Fire - Structure Deck: Blaze of Destructio...,$0.27,Indeterminate,24127,Fox Fire
16727,16591,9577,Grenosaurus,2 Level 3 monstersWhen this card destroys an o...,MONSTER,[Dinosaur／Xyz／Effect],FIRE,Rank 3,2000,1900,BATTLE PACK 3 MONSTER LEAGUE,BP03-EN116,2014-08-01,SH Shatterfoil,BP03-EN116SH,Grenosaurus (Shatterfoil) - Battle Pack 3: Mon...,$0.45,Indeterminate,91960,Grenosaurus
14086,13970,7690,Frequency Magician,"When this card is Normal Summoned, place 1 Spe...",MONSTER,[Spellcaster／Tuner／Effect],LIGHT,Level 2,800,400,5D's STARTER DECK 2008,5DS1-EN014,2008-08-05,C Common,5DS1-EN014C,Frequency Magician - 5D's 2008 Starter Deck (5...,$0.38,Indeterminate,32939,Frequency Magician


In [34]:
output_df[['name', 'description', 'set_id', 'rarity', 'price', 'volatility', 'type',
           'sub_type', 'attribute', 'rank', 'attack', 'defense', 'set_name', 'set_release',
           'name_official', 'index', 'index_market', 'join_id']].to_csv(f'yugioh-ccd-2025SEP12-163128.csv')